# 가설 정리
* 1. 서울시 내에서 스타벅스 매장이 어떻게 분포되어 있는지 확인
* 2. 일반 매장, 리저브, 드라이브스루 매장 분포에 어떤 차이가 있는지
*   위 두 가지 데이터 분석을 통해 아래 두 가설을 검증한다.
    * 가설 1: 거주 인구가 많은 지역
    * 가설 2: 직장인이 많은 지역 -> (산업별 밀집도)

그래서 할 일: (최대한 구체적으로 정리하기 ex사용할 시각화 라이브러리, 데이터셋 등)
0. 데이터셋 info(), describe(), corr(): 수치적으로 확인

1. 서울시 매장 분포 확인 : 
- 지도시각화 folium.Map : 서울시내 스타벅스 매장 위치 데이터(seoul_st_store_info) 이용, 지도에 위치 시각화 -> 몰려있는 곳 대략적으로 확인
- 바차트 sns.barplot(dodge=false) : 서울시내 스타벅스 매장 현황 수치 확실히 -> 지도시각화 보조

2. 가설 검증:

- 거주 인구가 많은 지역에 스타벅스 매장이 밀집되어 있다: 거주인구/매장 수 corr 수치 확인(산점도 scatter 그래프), 인구 당 매장 수 비율 확인(bar chart)

-> 거주 인구가 많은 지역?
- 직장인이 많은 지역에 스타벅스 매장이 밀집되어 있다: 종사자수/매장수 corr 수치 확인, 종사자수 당 매장수 비율 확인

+ 추가: 리저브 매장(부동산), WT 매장(유동인구 vs 직장인 vs 거주자), DT 매장(자동차 이동량)의 특성 

# 0. 데이터셋 info(), describe(), corr(): 수치적으로 확인

In [6]:
# 데이터 불러오기
import pandas as pd

seoul_st_store = pd.read_excel('./01.데이터수집/data_final/seoul_st_store_info.xlsx')
print(seoul_st_store.head())
print()
print(seoul_st_store.info())

       매장명         위도          경도         매장타입                          주소  \
0  역삼아레나빌딩  37.501087  127.043069  pin_general     서울특별시 강남구 언주로 425 (역삼동)   
1   논현역사거리  37.510178  127.022223  pin_general    서울특별시 강남구 강남대로 538 (논현동)   
2  신사역성일빌딩  37.513931  127.020606  pin_general    서울특별시 강남구 강남대로 584 (논현동)   
3   국기원사거리  37.499517  127.031495  pin_general    서울특별시 강남구 테헤란로 125 (역삼동)   
4  대치재경빌딩R  37.494668  127.062583  pin_reserve  서울특별시 강남구 남부순환로 2947 (대치동)   

        전화번호 시군구별  
0  1522-3232  강남구  
1  1522-3232  강남구  
2  1522-3232  강남구  
3  1522-3232  강남구  
4  1522-3232  강남구  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   매장명     608 non-null    object 
 1   위도      608 non-null    float64
 2   경도      608 non-null    float64
 3   매장타입    608 non-null    object 
 4   주소      608 non-null    object 
 5   전화번호    608 non-null    object 
 6   시군구별 

In [8]:
seoul_main = pd.read_excel('./01.데이터수집/data_final/seoul_st_pop_biz.xlsx')
print(seoul_main.head())
print()
print(seoul_main.info())

   시군구코드  시군구명         위도          경도  스타벅스_매장수  주민등록인구    종사자수   사업체수
0  11320   도봉구  37.665861  127.031767         6  312858   79097  25391
1  11380   은평구  37.617612  126.922700        14  468766  109031  36509
2  11230  동대문구  37.583801  127.050700        10  354884  146383  42813
3  11590   동작구  37.496504  126.944307        11  390377  109281  27915
4  11545   금천구  37.460097  126.900155        13  242467  255449  47964

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   시군구코드     25 non-null     int64  
 1   시군구명      25 non-null     object 
 2   위도        25 non-null     float64
 3   경도        25 non-null     float64
 4   스타벅스_매장수  25 non-null     int64  
 5   주민등록인구    25 non-null     int64  
 6   종사자수      25 non-null     int64  
 7   사업체수      25 non-null     int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 1.7+ KB
None


# 1. 서울시 매장 분포 확인 : 
- 지도시각화 folium.Map : 서울시내 스타벅스 매장 위치 데이터(seoul_st_store_info) 이용, 지도에 위치 시각화 -> 몰려있는 곳 대략적으로 확인
- 바차트 sns.barplot(dodge=false) : 서울시내 스타벅스 매장 현황 수치 확실히 -> 지도시각화 보조

## 지도시각화

### pre : 기본 지도 라이브러리 가져오기

In [9]:
import folium
starbucks_map = folium.Map(
    location=[37.573050,126.979189],
    zoom_start=12
)

starbucks_map

### 1. 스타벅스 지점 위치 정보 표기하기 : seoul_st_store 데이터프레임 활용

In [12]:
for idx in seoul_st_store.index:
    lat = seoul_st_store.loc[idx,'위도']        #.loc이 쓰였다.
    lng = seoul_st_store.loc[idx,'경도']
    
    folium.CircleMarker(
        location=[lat,lng],
        fill=True,
        fill_color='darkgreen',
        fill_opacity=0.9,
        color='darkseagreen',
        weight=1,
        radius=3
    ).add_to(starbucks_map)
starbucks_map

In [18]:
help(pd.DataFrame)


Help on class DataFrame in module pandas.core.frame:

class DataFrame(pandas.core.generic.NDFrame, pandas.core.arraylike.OpsMixin)
 |  DataFrame(data=None, index: 'Axes | None' = None, columns: 'Axes | None' = None, dtype: 'Dtype | None' = None, copy: 'bool | None' = None) -> 'None'
 |  
 |  Two-dimensional, size-mutable, potentially heterogeneous tabular data.
 |  
 |  Data structure also contains labeled axes (rows and columns).
 |  Arithmetic operations align on both row and column labels. Can be
 |  thought of as a dict-like container for Series objects. The primary
 |  pandas data structure.
 |  
 |  Parameters
 |  ----------
 |  data : ndarray (structured or homogeneous), Iterable, dict, or DataFrame
 |      Dict can contain Series, arrays, constants, dataclass or list-like objects. If
 |      data is a dict, column order follows insertion-order. If a dict contains Series
 |      which have an index defined, it is aligned by its index. This alignment also
 |      occurs if data i

#### 혼자 해보고 싶었던 것: folium.Map 위에 원형 도형으로 밀집 구역 표시하기

In [62]:
# # 혼자만의 희망 1 : 지도 위에 도형으로 밀집 구역 표시하기
# mean_lat = pd.Series(seoul_st_store.groupby('시군구별')['위도'].mean())
# mean_lng = pd.Series(seoul_st_store.groupby('시군구별')['경도'].mean())

# loc_df = pd.concat([mean_lat,mean_lng],axis=1)
# loc_df
# for_column = loc_df.index
# loc_df['시군구별'] = for_column
# loc_df

,위도,경도,시군구별
시군구별,,,
강남구,37.506952,127.045886,강남구
강동구,37.541943,127.140510,강동구
강북구,37.633118,127.023404,강북구
강서구,37.556407,126.838311,강서구
관악구,37.481485,126.946055,관악구
광진구,37.544898,127.081357,광진구
구로구,37.494102,126.879393,구로구
금천구,37.472950,126.889013,금천구
노원구,37.639370,127.065352,노원구


In [67]:
# for idx in loc_df['시군구별']:
#     lat = loc_df.loc[idx,'위도']
#     lng = loc_df.loc[idx,'경도']
#     # print('lat: ',lat)
#     # print('lng: ',lng)
#     folium.Circle(
#         location=[lat,lng],
#         radius=0.00000003,  # specify the radius in meters
#         color='black',
#         fill=True,
#         fill_color='black',
#         fill_opacity=0.2
#     ).add_to(starbucks_map)

In [68]:
# starbucks_map

#### 1-1. 지도 위에 스타벅스 매장 유형별로 다르게 표기하기